In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import contractions
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Embedding, LSTM, Dropout, GRU, Bidirectional, Input, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow.keras.backend as K

import os
from functools import reduce
import re
import json
import time
from datetime import date, datetime

In [2]:
np.random.seed(424242)

In [3]:
# GPU testing
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [4]:
df = pd.read_csv('./Documents/scripts/cleaned_corpora.csv')

# removes punctuation for simplicity
df.message = df.message.apply(contractions.fix)
df.message = df.message.apply(lambda m: re.sub(r"[\d:\.\?!,]", '', m).lower())
df.message = df.message.apply(lambda m: m.replace("'", ''))
df.message = df.message.apply(lambda m: re.sub(r" +", ' ', m))

df = df[df.message.apply(lambda m: len(m.split(' '))) >= 40]

In [5]:
alphabet = df.message.tolist()
alphabet = map(set, alphabet)
alphabet = reduce(lambda s, u: s.union(u), alphabet, set([]))

print(sorted(alphabet))

[' ', '-', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [7]:
df['date'] = pd.to_datetime(df.date)

YEARS = df.date.apply(lambda d: d.year).unique()

year_std = np.std(YEARS)
year_mean = np.mean(YEARS)

df['year'] = df.date.apply(lambda d: (d.year - year_mean) / year_std)

df['cos_month'] = df.date.apply(lambda d: np.cos(np.pi * 2 * d.month / 12))
df['sin_month'] = df.date.apply(lambda d: np.sin(np.pi * 2 * d.month / 12))

df['cos_day'] = df.date.apply(lambda d: np.cos(np.pi * 2 * d.day / 31))
df['sin_day'] = df.date.apply(lambda d: np.sin(np.pi * 2 * d.day / 31))

In [8]:
sign_dummies = pd.get_dummies(df.star_sign_index, prefix='star_sign')
df[sign_dummies.columns] = sign_dummies

In [9]:
df.head(2)

,date,message,source,star_sign_index,star_sign_name,year,cos_month,sin_month,cos_day,sin_day,...,star_sign_2,star_sign_3,star_sign_4,star_sign_5,star_sign_6,star_sign_7,star_sign_8,star_sign_9,star_sign_10,star_sign_11
0,2019-05-02,you may feel like you are being left out of th...,HoroscopeCom,0,Aries,1.303572,-0.866025,0.5,0.918958,0.394356,...,0,0,0,0,0,0,0,0,0,0
1,2019-05-03,is a romantic partner having trouble communica...,HoroscopeCom,0,Aries,1.303572,-0.866025,0.5,0.820763,0.571268,...,0,0,0,0,0,0,0,0,0,0


In [10]:
TRAINING_SIZE = 0.9

X_len = df.shape[0]
X_train_len = int(X_len * TRAINING_SIZE)

idx = np.random.random(df.shape[0]) < TRAINING_SIZE

X_train = df[idx]
X_test = df[~idx]

In [11]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)

X_train (42400, 22)
X_test (4702, 22)


In [12]:
embeddings_index = dict()
GLOVE_SIZE = 200

with open('./Documents/scripts/glove/glove.6B.{}d.txt'.format(GLOVE_SIZE), encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [13]:
max_words = 50000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train.message.values)
sequences = tokenizer.texts_to_sequences(X_train.message.values)

In [14]:
VOCABULARY_SIZE = len(tokenizer.word_index) + 1

In [15]:
EMBEDDING_INIT = np.zeros((VOCABULARY_SIZE, GLOVE_SIZE))

for (word, index) in tokenizer.word_index.items():
    if word not in embeddings_index:
        continue
        
    EMBEDDING_INIT[index] = embeddings_index[word]

In [16]:
STATIC_COLUMNS = [
    'year',
    
    'cos_month',
    'sin_month',
    
    'cos_day',
    'sin_day',
    
    'star_sign_0', 
    'star_sign_1', 
    'star_sign_2', 
    'star_sign_3', 
    'star_sign_4',
    'star_sign_5', 
    'star_sign_6', 
    'star_sign_7', 
    'star_sign_8',
    'star_sign_9', 
    'star_sign_10', 
    'star_sign_11',
]

In [17]:
TOKENS_BEFORE_PREDICTION = 30
TOKENS_PREDICTED = 1
STRIDE = TOKENS_BEFORE_PREDICTION

In [18]:
def get_sequence_sets(X):
    X_ = X.copy()
    
    seqs = np.array(tokenizer.texts_to_sequences(X.message.values)).reshape(-1, 1)
    X_ = X_[STATIC_COLUMNS].values
    
    X_ = np.concatenate([X_, seqs], axis=1)
    
    def slide_window(xs):
        return np.asarray([
            np.concatenate([
                xs[:-1], # static feats
                xs[-1][i * STRIDE : i * STRIDE + TOKENS_BEFORE_PREDICTION] #text
            ])
            for i in range(((len(xs[-1]) - TOKENS_BEFORE_PREDICTION) // STRIDE) + 1)
        ])

    X_ = np.asarray(list(map(slide_window, X_)))
    X_ = np.vstack(X_)
    
    sequences = X_[:, len(STATIC_COLUMNS):].astype(int)
    statics = X_[:, :len(STATIC_COLUMNS)].astype(float)
    
    y = sequences[:, TOKENS_PREDICTED:]
    sequences = sequences[:, :-TOKENS_PREDICTED]

#     y = tf.keras.utils.to_categorical(y, num_classes=VOCABULARY_SIZE, dtype='uint16')
    
    return sequences, statics, y

In [19]:
seq_x_train, static_x_train, seq_y_train = get_sequence_sets(X_train)
seq_x_test, static_x_test, seq_y_test = get_sequence_sets(X_test)

In [20]:
print(seq_x_train.shape)
print(static_x_train.shape)
print(seq_y_train.shape)

print(seq_x_test.shape)
print(static_x_test.shape)
print(seq_y_test.shape)

(88133, 29)
(88133, 17)
(88133, 29)
(9777, 29)
(9777, 17)
(9777, 29)


In [21]:
def create_model(batch_size=600, sequence_length=(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED)):
    static_input = Input(batch_shape=(batch_size, len(STATIC_COLUMNS),))
    seq_input = Input(batch_shape=(batch_size, sequence_length))

    sequential_model = Embedding(
        VOCABULARY_SIZE, 
        GLOVE_SIZE, 
        weights=[EMBEDDING_INIT], 
        input_length=sequence_length,
        trainable=False,
        name='sequence_embedding',
    )(seq_input)

    sequential_model = (LSTM(1000, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))(sequential_model)
    sequential_model = (LSTM(500, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))(sequential_model)


    sequential_model = Model(inputs=seq_input, outputs=sequential_model)


    static_model = Dense(100, activation='tanh')(static_input)
    static_model = Dropout(0.1)(static_model)
    static_model = RepeatVector(sequence_length)(static_model)

    static_model = Model(inputs=static_input, outputs=static_model)

    combined = tf.keras.layers.concatenate(sequential_model.outputs + static_model.outputs, axis=2)

    final_model = Dense(VOCABULARY_SIZE // 5)(combined)
    final_model = Dense(VOCABULARY_SIZE)(combined)

    return Model(inputs=sequential_model.inputs + static_model.inputs, outputs=final_model)

In [22]:
BATCH_SIZE = 700

TRAIN_NEW_BATCHED_SIZE = (seq_x_train.shape[0] - (seq_x_train.shape[0] % BATCH_SIZE))
TEST_NEW_BATCHED_SIZE = (seq_x_test.shape[0] - (seq_x_test.shape[0] % BATCH_SIZE))

In [23]:
model_v2 = create_model(batch_size=BATCH_SIZE)    

In [24]:
model_v2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(700, 29)]          0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(700, 17)]          0                                            
__________________________________________________________________________________________________
sequence_embedding (Embedding)  (700, 29, 200)       4265200     input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (700, 100)           1800        input_1[0][0]                    
____________________________________________________________________________________________

In [42]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

tf.keras.optimizers.Adam(learning_rate=0.01)

model_v2.compile(optimizer='adam', loss=loss)

In [43]:
now = date.today().strftime('%Y.%m.%d.') + str(int(time.time()))

csv_logger = CSVLogger('./Documents/scripts/logs/train.%s.csv' % now, append=True)

es = EarlyStopping(mode='min', patience=15, monitor='loss')

mc = ModelCheckpoint(
    './Documents/scripts/models/best_model.%s.weights' % now, 
    mode='min', 
    save_best_only=True,
    save_weights_only=True,
    monitor='loss'
)

lrr = ReduceLROnPlateau(factor=0.2, min_lr=1e-4, patience=10, monitor='loss')

In [44]:
now

'2020.06.07.1591567376'

In [45]:
history = model_v2.fit(
    (seq_x_train[:TRAIN_NEW_BATCHED_SIZE], static_x_train[:TRAIN_NEW_BATCHED_SIZE]),
    seq_y_train[:TRAIN_NEW_BATCHED_SIZE],
    epochs = 80,
    batch_size = BATCH_SIZE,
#     validation_data=(
#         (seq_x_test[:TEST_NEW_BATCHED_SIZE], static_x_test[:TEST_NEW_BATCHED_SIZE]), 
#         seq_y_test[:TEST_NEW_BATCHED_SIZE]
#     ),
     callbacks=[csv_logger, es, mc, lrr],
)

Train on 87500 samples
Epoch 1/80
87500/87500 [==============================] - 80s 918us/sample - loss: 5.4515
Epoch 2/80
87500/87500 [==============================] - 79s 899us/sample - loss: 5.2086
Epoch 3/80
87500/87500 [==============================] - 79s 898us/sample - loss: 5.0231
Epoch 4/80
87500/87500 [==============================] - 79s 899us/sample - loss: 4.8777
Epoch 5/80
87500/87500 [==============================] - 79s 898us/sample - loss: 4.7549
Epoch 6/80
87500/87500 [==============================] - 79s 898us/sample - loss: 4.6534
Epoch 7/80
87500/87500 [==============================] - 79s 903us/sample - loss: 4.5617
Epoch 8/80
87500/87500 [==============================] - 79s 901us/sample - loss: 4.4835
Epoch 9/80
87500/87500 [==============================] - 79s 900us/sample - loss: 4.4165
Epoch 10/80
87500/87500 [==============================] - 79s 901us/sample - loss: 4.3543
Epoch 11/80
87500/87500 [==============================] - 79s 901us/sample 

In [47]:
model = create_model(batch_size=1)
model.load_weights('./Documents/scripts/models/best_model.2020.06.07.1591567376.weights')
model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(1, 29)]            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(1, 17)]            0                                            
__________________________________________________________________________________________________
sequence_embedding (Embedding)  (1, 29, 200)         4265200     input_6[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (1, 100)             1800        input_5[0][0]                    
____________________________________________________________________________________________

In [49]:
STAR_SIGN_NAMES = df[['star_sign_index', 'star_sign_name']].drop_duplicates().to_dict(orient='records')

In [52]:
def continue_text(model, text_sequence, predicted_size, star_sign, year, month, day):
    if isinstance(star_sign, str):
        star_sign = list(filter(lambda x: x['star_sign_name'].lower() == star_sign.lower(), STAR_SIGN_NAMES))[0]['star_sign_index']
    
    tf.keras.backend.set_floatx('float64')
    
    tokenized_text = tokenizer.texts_to_sequences([text_sequence])
    predicted_size = predicted_size + len(tokenized_text[0])
    
    year = (year - year_mean) / year_std
    cos_mon = np.cos(np.pi * 2 * month / 12.0)
    sin_mon = np.cos(np.pi * 2 * month / 12.0)
    cos_day = np.cos(np.pi * 2 * day / 31.0)
    sin_day = np.cos(np.pi * 2 * day / 31.0)
    
    star_signs = np.zeros(12)
    star_signs[star_sign] = 1
    
    static_feats = np.array([year, cos_mon, sin_mon, cos_day, sin_day])
    static_feats = np.concatenate([static_feats, star_signs])
    
    model.reset_states()
    temperature = 1
    
    tf.keras.backend.set_floatx('float64')
    
    while len(tokenized_text[0]) < predicted_size:
        padded_sentence = pad_sequences(
            tokenized_text[-(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED):], 
            maxlen=(TOKENS_BEFORE_PREDICTION - TOKENS_PREDICTED)
        )
        
        op = model([np.asarray(padded_sentence).reshape(1, -1), static_feats.reshape(1, -1)])
        predictions = tf.squeeze(op, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        tokenized_text[0].append(predicted_id)
      
    return ' '.join(map(lambda x: dict(map(reversed, tokenizer.word_index.items()))[x], tokenized_text[0]))





In [57]:
continue_text(model, "", 30, 'Taurus', 2019, 1, 13)

'with the product of their materials materials create an amazing world of how much other information could be a tedious exercise be slightly emotionally you have a very useful period'

In [136]:
continue_text(model, "Today", 20, 'Aries', 2019, 1, 14)

'today rattles oneonone hosted streamlining hosted hosted streamlining hosted hosted invincibility authoritarianism hosted allconsuming jail hosted hosted manuscript invincibility oven wednesday'

In [ ]:
with open('./Documents/scripts/dictionary.2020.05.03.1.json', 'w') as f:
    f.write(json.dumps(list(tokenizer.word_index.keys())))